In [1]:
import os
import requests
import feedparser
import re
import numpy
from datetime import datetime, timezone
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline
from keybert.llm import TextGeneration
from keybert import KeyLLM
from sentence_transformers import SentenceTransformer

endpoint = "https://active-monitor-48.hasura.app/v1/graphql"
admin_key = "bAQuK7HSYvMAp6S6pnqXH0wQlyuKNUICzoW3jwecc27pwz6COLhE750s5YAec7Hz"


def query_hasura_graphql(endpoint, admin_key, query, variables):
    headers = {
        'Content-Type': 'application/json',
        'x-hasura-admin-secret': f'{admin_key}'
    }

    data = {
        'query': query,
        'variables': variables
    }
    response = requests.post(endpoint, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Request failed with status code {response.status_code}")
        return None

def is_valid_timezone_format(published):
    try:
        # Attempt to parse the string
        date_format = "%a, %d %b %Y %H:%M:%S %z"
        date_object = datetime.strptime(published, date_format)

        hasura_timestamp = date_object.astimezone(timezone.utc).isoformat()
        return True, hasura_timestamp
    except ValueError:
        # If parsing fails, the string is not in the correct format
        return False, None

def check_date_format(date_string):
    try:
        datetime.strptime(date_string, '%Y-%m-%dT%H:%M:%S%z')
        return True
    except ValueError:
        return False

def mutation_hasura_graphql(endpoint, admin_key, mutation_query, mutation_variables):
    headers = {
        'Content-Type': 'application/json',
        'x-hasura-admin-secret': f'{admin_key}'
    }
    response = requests.post(endpoint, json={'query': mutation_query, 'variables': mutation_variables}, headers=headers)
    if response.ok:
        data = response.json()
        print(data)
        return True, data
    else:
        print(f"Mutation failed with status code {response.status_code}: {response.text}")
        return False, None


In [19]:
def array2dto2d(n2):
    n1 = []
    for sublist in n2:
        for element in sublist:
            n1.append(element)
    n2 = list(set(n1))
    return n2
def detail_tags(offset1):
  graphql_query = '''
  query MyQuery($limit: Int!, $offset: Int!) {
  synopse_articles_t_v4_article_groups_l2_detail(where: {logo_urls: {_is_null: true}, summary_60_words: {_is_null: false}}, limit: $limit, offset: $offset) {
      id
      t_v3_article_groups_l2 {
        articles_group
      }
    }
  }
  '''
  query2 = '''
  query MyQuery($article_id: bigint = "") {
    synopse_articles_t_v1_rss1_articles(where: {id: {_eq: $article_id}}) {
      image_link
      tags
      t_v1_rss1_feed_link {
        t_v1_outlet {
          logo_url
        }
      }
      t_v2_articles_summary {
        keywords_tags
        location_tags
        org_tags
        person_tags
      }
    }
  }
  '''
  offset = offset1
  mutation_query = """
  mutation MyMutation($updates: [synopse_articles_t_v4_article_groups_l2_detail_updates!] = {where: {}}) {
    update_synopse_articles_t_v4_article_groups_l2_detail_many(updates: $updates) {
      affected_rows
    }
  }
  """
  while True:
      variables = {
      "limit": 1,
      "offset": offset
      }
      response_data = query_hasura_graphql(endpoint, admin_key, graphql_query, variables)
      synopse_articles_t_v4_article_groups_l2_detail_updates_loc=[]
      image_urls=[]
      logo_urls=[]
      keywords_tags=[]
      location_tags=[]
      org_tags=[]
      person_tags=[]
      if len(response_data['data']['synopse_articles_t_v4_article_groups_l2_detail']) == 0:
            break
      for response in response_data['data']['synopse_articles_t_v4_article_groups_l2_detail']:
          for article in response['t_v3_article_groups_l2']['articles_group']:
              variables2 = {
              "article_id": article
              }
              response_data1 = query_hasura_graphql(endpoint, admin_key, query2, variables2)
              for response1 in response_data1['data']['synopse_articles_t_v1_rss1_articles']:
                  image_urls.append(response1['image_link'])
                  logo_urls.append(response1['t_v1_rss1_feed_link']['t_v1_outlet']['logo_url'])
                  keywords_tags.append(response1['tags'])
                  keywords_tags.append(response1['t_v2_articles_summary']['keywords_tags'])
                  location_tags.append(response1['t_v2_articles_summary']['location_tags'])
                  org_tags.append(response1['t_v2_articles_summary']['org_tags'])
                  person_tags.append(response1['t_v2_articles_summary']['person_tags'])
      image_urls = list(set(image_urls))
      logo_urls = list(set(logo_urls))
      keywords_tags = array2dto2d(keywords_tags)
      location_tags = array2dto2d(location_tags)
      org_tags = array2dto2d(org_tags)
      person_tags = array2dto2d(person_tags)
      synopse_articles_t_v4_article_groups_l2_detail_updates_loc.append({
          "where": {"id" : { "_eq": response['id'] }},
          "_set": {"image_urls": image_urls,
                  "logo_urls": logo_urls,
                  "keywords_tags": keywords_tags,
                  "location_tags": location_tags,
                  "org_tags": org_tags,
                  "person_tags": person_tags }
          })
      mutation_variables = {
      "updates": synopse_articles_t_v4_article_groups_l2_detail_updates_loc,
      }
      out1 = mutation_hasura_graphql(endpoint=endpoint, admin_key=admin_key, mutation_query=mutation_query, mutation_variables=mutation_variables)

detail_tags(5)

In [17]:
graphql_query = '''
query MyQuery($limit: Int!, $offset: Int!) {
  synopse_articles_t_v3_article_groups_l1(where: {t_v1_rss1_article: {is_grouped: {_eq: 1}}, article_count: {_gt: 1}}, order_by: {updated_at: asc}, limit: $limit, offset: $offset) {
    article_id
    initial_group
  }
}
'''
query2 = '''
query MyQuery($articleid: [bigint!] = []) {
    synopse_articles_t_v3_article_groups_l1(where: {initial_group: {_contains: $articleid}}) {
    article_id
    initial_group
    }
}
'''
query3 = '''
query MyQuery($articleid: [bigint!] = []) {
    synopse_articles_t_v3_article_groups_l2(where: {articles_group: {_contains: $articleid}}, order_by: {is_valid: desc, updated_at: desc}) {
    id
    articles_group
    is_valid
    }
}
'''
mutation_query = """
mutation MyMutation($objects: [synopse_articles_t_v3_article_groups_l2_insert_input!] = {}, $updates: [synopse_articles_t_v1_rss1_articles_updates!] = {where: {}}, $updates1: [synopse_articles_t_v3_article_groups_l2_updates!] = {where: {}}, $articleGroupIds: [bigint!] = "") {
  insert_synopse_articles_t_v3_article_groups_l2(objects: $objects, on_conflict: {constraint: t_v3_article_groups_l2_pkey}) {
    affected_rows
  }
  update_synopse_articles_t_v1_rss1_articles_many(updates: $updates) {
    affected_rows
  }
  update_synopse_articles_t_v3_article_groups_l2_many(updates: $updates1) {
    affected_rows
  }
  delete_synopse_articles_t_v3_article_groups_l2(where: {id: {_in: $articleGroupIds}}) {
    affected_rows
  }
}
"""
while True:
    variables = {
    "limit": 1,
    "offset": 0
    }
    response_data = query_hasura_graphql(endpoint, admin_key, graphql_query, variables)
    synopse_articles_t_v3_article_groups_l2_insert_input_loc=[]
    synopse_articles_t_v1_rss1_articles_updates_loc=[]
    synopse_articles_t_v3_article_groups_l2_updates_loc=[]
    articleGroupIds_loc=[]
    if len(response_data['data']['synopse_articles_t_v3_article_groups_l1']) == 0:
        break
    for response in response_data['data']['synopse_articles_t_v3_article_groups_l1']:
        variables2 = {
            "articleid": [response['article_id']]
            }
        response_data1 = query_hasura_graphql(endpoint, admin_key, query2, variables2)
        articles_ids = []
        if len(response_data1['data']['synopse_articles_t_v3_article_groups_l1']) > 0:
            for func_response in response_data1['data']['synopse_articles_t_v3_article_groups_l1']:
                articles_ids.append(func_response['initial_group'])
        n1 = []
        for sublist in articles_ids:
            for element in sublist:
                n1.append(element)
        articles_ids = list(set(n1))
        articles_ids.sort(reverse=True)
        while True:
            if len(articles_ids) > 1:
                articles_news = []
                for article_id in articles_ids:
                    variables3 = {
                        "articleid": [article_id]
                        }
                    response_data2 = query_hasura_graphql(endpoint, admin_key, query2, variables3)
                    if len(response_data1['data']['synopse_articles_t_v3_article_groups_l1']) > 0:
                        for func_response in response_data1['data']['synopse_articles_t_v3_article_groups_l1']:
                            articles_news.append(func_response['initial_group'])
                n2 = []
                for sublist in articles_news:
                    for element in sublist:
                        n2.append(element)
                articles_news = list(set(n2))
                articles_news.sort(reverse=True)
                if articles_ids == articles_news:
                    break
                else:
                    articles_ids = articles_news
        n5 = []
        for a1 in articles_ids:
            variables3 = {
                "articleid": [a1]
                }
            response_data2 = query_hasura_graphql(endpoint, admin_key, query3, variables3)
            if len(response_data2['data']['synopse_articles_t_v3_article_groups_l2']) > 0:
                for func_response in response_data2['data']['synopse_articles_t_v3_article_groups_l2']:
                    n5.append(func_response['articles_group'])
                    if func_response('is_valid') == 0:
                        articleGroupIds_loc.append(func_response['id'])
        n5.append(articles_ids)
        n2 = []
        for sublist in n5:
            for element in sublist:
                n2.append(element)
        articles_ids = list(set(n2))
        articles_ids.sort(reverse=True)
        synopse_articles_t_v3_article_groups_l2_insert_input_loc.append({
            "articles_group": articles_ids,
            'articles_in_group': len(articles_ids)
            }
            )
        for article in articles_ids:
            synopse_articles_t_v1_rss1_articles_updates_loc.append({
                "where": {"id" : { "_eq": article }},
                "_set": {"is_grouped": 2}
                })
    mutation_variables = {
        "objects": synopse_articles_t_v3_article_groups_l2_insert_input_loc,
        "updates": synopse_articles_t_v1_rss1_articles_updates_loc,
        "updates1": synopse_articles_t_v3_article_groups_l2_updates_loc,
        "articleGroupIds": articleGroupIds_loc
        }
    out1 = mutation_hasura_graphql(endpoint=endpoint, admin_key=admin_key, mutation_query=mutation_query, mutation_variables=mutation_variables)
            
        

In [14]:
graphql_query = '''
query MyQuery($limit: Int!, $offset: Int!) {
  synopse_articles_t_v3_article_groups_l1(where: {t_v1_rss1_article: {is_grouped: {_eq: 1}}, article_count: {_gt: 1}}, order_by: {updated_at: asc}, limit: $limit, offset: $offset) {
    article_id
    initial_group
  }
}
'''
query2 = '''
query MyQuery($articleid: [bigint!] = []) {
    synopse_articles_t_v3_article_groups_l1(where: {initial_group: {_contains: $articleid}}) {
    article_id
    initial_group
    }
}
'''
query3 = '''
query MyQuery($articleid: [bigint!] = []) {
    synopse_articles_t_v3_article_groups_l2(where: {articles_group: {_contains: $articleid}, is_valid: {_eq: 0}}, order_by: {is_valid: desc, updated_at: desc}, limit: 1) {
    id
    articles_group
    is_summerized
    is_valid
    }
}
'''
mutation_query = """
mutation MyMutation($objects: [synopse_articles_t_v3_article_groups_l2_insert_input!] = {}, $updates: [synopse_articles_t_v1_rss1_articles_updates!] = {where: {}}, $updates1: [synopse_articles_t_v3_article_groups_l2_updates!] = {where: {}}, $articleGroupIds: [bigint!] = "") {
  insert_synopse_articles_t_v3_article_groups_l2(objects: $objects, on_conflict: {constraint: t_v3_article_groups_l2_pkey}) {
    affected_rows
  }
  update_synopse_articles_t_v1_rss1_articles_many(updates: $updates) {
    affected_rows
  }
  update_synopse_articles_t_v3_article_groups_l2_many(updates: $updates1) {
    affected_rows
  }
  delete_synopse_articles_t_v3_article_groups_l2(where: {id: {_in: $articleGroupIds}}) {
    affected_rows
  }
}
"""
variables = {
"limit": 20,
"offset": 0
}
response_data = query_hasura_graphql(endpoint, admin_key, graphql_query, variables)
synopse_articles_t_v3_article_groups_l2_insert_input_loc=[]
synopse_articles_t_v1_rss1_articles_updates_loc=[]
synopse_articles_t_v3_article_groups_l2_updates_loc=[]
articleGroupIds=[]
for response in response_data['data']['synopse_articles_t_v3_article_groups_l1']:
    variables2 = {
        "articleid": [response['article_id']]
        }
    response_data1 = query_hasura_graphql(endpoint, admin_key, query2, variables2)
    articles_ids = []
    if len(response_data1['data']['synopse_articles_t_v3_article_groups_l1']) > 0:
        for func_response in response_data1['data']['synopse_articles_t_v3_article_groups_l1']:
            articles_ids.append(func_response['initial_group'])
    n1 = []
    for sublist in articles_ids:
        for element in sublist:
            n1.append(element)
    articles_ids = list(set(n1))
    articles_ids.sort(reverse=True)
    while True:
        if len(articles_ids) > 1:
            articles_news = []
            for article_id in articles_ids:
                variables3 = {
                    "articleid": [article_id]
                    }
                response_data2 = query_hasura_graphql(endpoint, admin_key, query2, variables3)
                if len(response_data1['data']['synopse_articles_t_v3_article_groups_l1']) > 0:
                    for func_response in response_data1['data']['synopse_articles_t_v3_article_groups_l1']:
                        articles_news.append(func_response['initial_group'])
            n2 = []
            for sublist in articles_news:
                for element in sublist:
                    n2.append(element)
            articles_news = list(set(n2))
            articles_news.sort(reverse=True)
            if articles_ids == articles_news:
                break
            else:
                articles_ids = articles_news
     
    last_element = articles_ids[-1]
    variables3 = {
        "articleid": [last_element]
        }
    response_data2 = query_hasura_graphql(endpoint, admin_key, query3, variables3)
    if len (response_data2['data']['synopse_articles_t_v3_article_groups_l2'] > 0):
        for func_response in response_data2['data']['synopse_articles_t_v3_article_groups_l2']:
            articles_ids.append(func_response['articles_group'])
            articleGroupIds.append(func_response['id'])
        n2 = []
        for sublist in articles_ids:
            for element in sublist:
                n2.append(element)
        articles_ids = list(set(n2))
        articles_ids.sort(reverse=True)
    synopse_articles_t_v3_article_groups_l2_insert_input_loc.append({
        "articles_group": articles_ids,
        'articles_in_group': len(articles_ids)
        }
        )
    for article in articles_ids:
        synopse_articles_t_v1_rss1_articles_updates_loc.append({
            "where": {"id" : { "_eq": article }},
            "_set": {"is_grouped": 2}
            })

[504, 441]
441
{'data': {'synopse_articles_t_v3_article_groups_l2': []}}
[515, 275]
275
{'data': {'synopse_articles_t_v3_article_groups_l2': []}}
[442, 30]
30
{'data': {'synopse_articles_t_v3_article_groups_l2': []}}
[500, 305, 284, 109]
109
{'data': {'synopse_articles_t_v3_article_groups_l2': []}}
[504, 441]
441
{'data': {'synopse_articles_t_v3_article_groups_l2': []}}
[448, 270]
270
{'data': {'synopse_articles_t_v3_article_groups_l2': []}}
[413, 51]
51
{'data': {'synopse_articles_t_v3_article_groups_l2': []}}
[287, 77, 61]
61
{'data': {'synopse_articles_t_v3_article_groups_l2': []}}
[290, 64]
64
{'data': {'synopse_articles_t_v3_article_groups_l2': []}}
[362, 293]
293
{'data': {'synopse_articles_t_v3_article_groups_l2': []}}
[312, 125]
125
{'data': {'synopse_articles_t_v3_article_groups_l2': []}}
[299, 282]
282
{'data': {'synopse_articles_t_v3_article_groups_l2': []}}
[347, 319, 316, 119]
119
{'data': {'synopse_articles_t_v3_article_groups_l2': []}}
[515, 275]
275
{'data': {'synopse_a

In [ ]:
graphql_query = '''
query MyQuery($limit: Int!, $offset: Int!) {
  synopse_articles_t_v3_article_groups_l1(where: {t_v1_rss1_article: {is_grouped: {_eq: 1}}, article_count: {_gt: 1}}, order_by: {updated_at: asc}, limit: $limit, offset: $offset) {
    article_id
    initial_group
  }
}
'''
query2 = '''
query MyQuery($articleid: [bigint!] = []) {
    synopse_articles_t_v3_article_groups_l1(where: {initial_group: {_contains: $articleid}}) {
    article_id
    initial_group
    }
}
'''
query3 = '''
query MyQuery($articleid: [bigint!] = []) {
    synopse_articles_t_v3_article_groups_l2(where: {articles_group: {_contains: $articleid}, is_valid: {_eq: 0}}, order_by: {is_valid: desc, updated_at: desc}, limit: 1) {
    id
    articles_group
    is_summerized
    is_valid
    }
}
'''
mutation_query = """
    mutation MyMutation($objects: [synopse_articles_t_v3_article_groups_l2_insert_input!] = {}, $updates: [synopse_articles_t_v1_rss1_articles_updates!] = {where: {}}, $updates1: [synopse_articles_t_v3_article_groups_l2_updates!] = {where: {}}) {
        insert_synopse_articles_t_v3_article_groups_l2(objects: $objects, on_conflict: {constraint: t_v3_article_groups_l2_pkey}) {
        affected_rows
        }
        update_synopse_articles_t_v1_rss1_articles_many(updates: $updates) {
        affected_rows
        }
        update_synopse_articles_t_v3_article_groups_l2_many(updates: $updates1) {
        affected_rows
        }
    }
"""
variables = {
"limit": 2,
"offset": 0
}
response_data = query_hasura_graphql(endpoint, admin_key, graphql_query, variables)
for response in response_data['data']['synopse_articles_t_v3_article_groups_l1']:
    print(response)
    variables2 = {
        "articleid": [response['article_id']]
        }
    synopse_articles_t_v3_article_groups_l2_insert_input_loc=[]
    synopse_articles_t_v1_rss1_articles_updates_loc=[]
    synopse_articles_t_v3_article_groups_l2_updates_loc=[]
    response_data1 = query_hasura_graphql(endpoint, admin_key, query2, variables2)
    articles_ids = []
    if len(response_data1['data']['synopse_articles_t_v3_article_groups_l1']) > 0:
        for func_response in response_data1['data']['synopse_articles_t_v3_article_groups_l1']:
            articles_ids.append(func_response['initial_group'])
    n1 = []
    for sublist in articles_ids:
        for element in sublist:
            n1.append(element)
    articles_ids = list(set(n1))
    articles_ids.sort(reverse=True)
    print(articles_ids)
    while True:
        if len(articles_ids) > 1:
            articles_news = []
            for article_id in articles_ids:
                variables3 = {
                    "articleid": [article_id]
                    }
                response_data2 = query_hasura_graphql(endpoint, admin_key, query2, variables3)
                if len(response_data1['data']['synopse_articles_t_v3_article_groups_l1']) > 0:
                    for func_response in response_data1['data']['synopse_articles_t_v3_article_groups_l1']:
                        articles_news.append(func_response['initial_group'])
            n2 = []
            for sublist in articles_news:
                for element in sublist:
                    n2.append(element)
            articles_news = list(set(n2))
            articles_news.sort(reverse=True)
            if articles_ids == articles_news:
                print(articles_news)
                break
            else:
                articles_ids = articles_news
    exists = 0
    ids = []
    articles_groups = []
    for article_id in articles_ids:
        variables4 = {
        "articleid": [article_id]
        }
        response_data3 = query_hasura_graphql(endpoint, admin_key, query3, variables4)
        if len(response_data3['data']['synopse_articles_t_v3_article_groups_l2']) > 0:
            for func_response in response_data3['data']['synopse_articles_t_v3_article_groups_l2']:
                if func_response['is_valid'] == 0:
                    exists = exists + 1
                    ids.append(func_response['id'])
                    articles_groups.append(func_response['articles_group'])
                
                    
    if exists == 0 : 
        synopse_articles_t_v3_article_groups_l2_insert_input_loc.append({
            "articles_group": articles_ids,
            'articles_in_group': len(articles_ids)
            }
            )
        synopse_articles_t_v1_rss1_articles_updates_loc.append({
            "where": {"id" : { "_eq": response['id'] }},
            "_set": {"is_grouped": 2}
            })

In [17]:
graphql_query = '''
query MyQuery($limit: Int!, $offset: Int!) {
  synopse_articles_t_v3_article_groups_l1(where: {t_v1_rss1_article: {is_grouped: {_eq: 1}}, article_count: {_gt: 1}}, order_by: {updated_at: asc}, limit: $limit, offset: $offset) {
    article_id
    initial_group
  }
}
'''
query2 = '''
query MyQuery($articleid: [bigint!] = [1088]) {
  synopse_articles_t_v3_article_groups_l1(where: {initial_group: {_contains: $articleid}}) {
    initial_group
  }
}
'''
mutation_query = """
mutation MyMutation($objects: [synopse_articles_t_v3_article_groups_l2_insert_input!] = {}, $updates: [synopse_articles_t_v1_rss1_articles_updates!] = {where: {}}, $updates1: [synopse_articles_t_v3_article_groups_l2_updates!] = {where: {}}) {
    insert_synopse_articles_t_v3_article_groups_l2(objects: $objects, on_conflict: {constraint: t_v3_article_groups_l2_pkey}) {
    affected_rows
    }
    update_synopse_articles_t_v1_rss1_articles_many(updates: $updates) {
    affected_rows
      }
      update_synopse_articles_t_v3_article_groups_l2_many(updates: $updates1) {
      affected_rows
      }
}
"""
while True: 
    variables = {
    "limit": 1,
    "offset": 0
    }
    synopse_articles_t_v3_article_groups_l2_insert_input_loc=[]
    synopse_articles_t_v1_rss1_articles_updates_loc=[]
    synopse_articles_t_v3_article_groups_l2_updates_loc=[]
    response_data = query_hasura_graphql(endpoint, admin_key, graphql_query, variables)
    if len(response_data['data']['synopse_articles_t_v3_article_groups_l1']) == 0:
        break
    for response in response_data['data']['synopse_articles_t_v3_article_groups_l1']:
        print(response)
        variables2 = {
            "articleid": [response['article_id']]
            }
        func_response_data = query_hasura_graphql(endpoint, admin_key, query2, variables2)
        articles_ids = []
        if len(func_response_data['data']['synopse_articles_t_v3_article_groups_l1']) > 0:
            for func_response in func_response_data['data']['synopse_articles_t_v3_article_groups_l1']:
                articles_ids.append(func_response['initial_group'])
        new_lst = []
        for sublist in articles_ids:
            for element in sublist:
                new_lst.append(element)
        articles_ids = list(set(new_lst))
        
        print(articles_ids)
    break

{'article_id': 502, 'initial_group': [502, 458, 416, 105]}
[416, 105, 458, 502]


In [2]:
graphql_query = '''
query MyQuery($limit: Int!, $offset: Int!) {
  synopse_articles_t_v3_article_groups_l1(where: {t_v1_rss1_article: {is_grouped: {_eq: 1}}, article_count: {_gt: 1}}, order_by: {updated_at: asc}, limit: $limit, offset: $offset) {
    article_id
    initial_group
  }
}
'''
variables = {
"limit": 100,
"offset": 0
}
response_data = query_hasura_graphql(endpoint, admin_key, graphql_query, variables)
arrays = []
for response in response_data['data']['synopse_articles_t_v3_article_groups_l1']:
    print(response['initial_group'])
    arrays.append(response['initial_group'])

[502, 458, 416, 105]
[515, 275]
[504, 441]
[500, 284, 305]
[504, 441]
[502, 458]
[449, 285, 108]
[443, 285, 296, 109, 94]
[442, 30]
[448, 270]
[413, 51]
[502, 416]
[403, 109]
[282, 299]
[287, 77, 61]
[290, 64]
[293, 362]
[312, 125]
[283, 285]
[443, 285, 296, 94]
[316, 319, 347]
[500, 284, 305, 109]
[285, 349]
[280, 310]
[515, 275]
[316, 347]
[320, 295]
[443, 449, 283, 285, 349, 296, 75, 94]
[319, 347, 119]
[448, 270]
[280, 310]
[293, 362]
[318, 84]
[320, 295]
[282, 299]
[318, 84, 53]
[116, 108]
[319, 119]
[443, 285, 296, 94]
[500, 284, 305]
[109, 111]
[502, 105]
[285, 75]
[443, 403, 284, 109, 111]
[287, 77]
[449, 116, 108]
[312, 125]
[290, 64]
[84, 53]
[287, 61]
[413, 51]
[442, 30]


In [3]:
groups = {}
for array in arrays:
    for num in array:
        if num in groups:
            if array not in groups[num]:
                groups[num].append(array)
        else:
            groups[num] = [array]

# Remove duplicates
for key in groups:
    groups[key] = [list(x) for x in set(tuple(x) for x in groups[key])]

print(groups)

{502: [[502, 458], [502, 105], [502, 416], [502, 458, 416, 105]], 458: [[502, 458], [502, 458, 416, 105]], 416: [[502, 416], [502, 458, 416, 105]], 105: [[502, 105], [502, 458, 416, 105]], 515: [[515, 275]], 275: [[515, 275]], 504: [[504, 441]], 441: [[504, 441]], 500: [[500, 284, 305], [500, 284, 305, 109]], 284: [[500, 284, 305], [443, 403, 284, 109, 111], [500, 284, 305, 109]], 305: [[500, 284, 305], [500, 284, 305, 109]], 449: [[449, 116, 108], [443, 449, 283, 285, 349, 296, 75, 94], [449, 285, 108]], 285: [[285, 349], [285, 75], [443, 285, 296, 109, 94], [443, 449, 283, 285, 349, 296, 75, 94], [283, 285], [443, 285, 296, 94], [449, 285, 108]], 108: [[449, 116, 108], [116, 108], [449, 285, 108]], 443: [[443, 285, 296, 109, 94], [443, 449, 283, 285, 349, 296, 75, 94], [443, 403, 284, 109, 111], [443, 285, 296, 94]], 296: [[443, 285, 296, 109, 94], [443, 449, 283, 285, 349, 296, 75, 94], [443, 285, 296, 94]], 109: [[403, 109], [109, 111], [443, 285, 296, 109, 94], [443, 403, 284, 109

In [3]:
graphql_query = '''
query MyQuery($limit: Int!, $offset: Int!) {
  synopse_articles_t_v3_article_groups_l1(where: {t_v1_rss1_article: {is_grouped: {_eq: 1}}, article_count: {_gt: 1}}, order_by: {updated_at: asc}, limit: $limit, offset: $offset) {
    article_id
    initial_group
  }
}
'''
query2 = '''
query MyQuery($articleid: [bigint!] = []) {
    synopse_articles_t_v3_article_groups_l2(where: {articles_group: {_contains: $articleid}, is_valid: {_eq: 0}}, order_by: {updated_at: desc}, limit: 1) {
    id
    articles_group
    is_summerized
    is_valid
    }
}
'''
mutation_query = """
mutation MyMutation($objects: [synopse_articles_t_v3_article_groups_l2_insert_input!] = {}, $updates: [synopse_articles_t_v1_rss1_articles_updates!] = {where: {}}, $updates1: [synopse_articles_t_v3_article_groups_l2_updates!] = {where: {}}) {
    insert_synopse_articles_t_v3_article_groups_l2(objects: $objects, on_conflict: {constraint: t_v3_article_groups_l2_pkey}) {
    affected_rows
    }
    update_synopse_articles_t_v1_rss1_articles_many(updates: $updates) {
    affected_rows
    }
    update_synopse_articles_t_v3_article_groups_l2_many(updates: $updates1) {
    affected_rows
    }
}
"""
while True: 
    variables = {
    "limit": 1,
    "offset": 0
    }
    synopse_articles_t_v3_article_groups_l2_insert_input_loc=[]
    synopse_articles_t_v1_rss1_articles_updates_loc=[]
    synopse_articles_t_v3_article_groups_l2_updates_loc=[]
    response_data = query_hasura_graphql(endpoint, admin_key, graphql_query, variables)
    if len(response_data['data']['synopse_articles_t_v3_article_groups_l1']) == 0:
        break
    for response in response_data['data']['synopse_articles_t_v3_article_groups_l1']:
        print(response)
        variables2 = {
          "articleid": [response['article_id']]
          }
        response_data1 = query_hasura_graphql(endpoint, admin_key, query2, variables2)
        if len(response_data1['data']['synopse_articles_t_v3_article_groups_l2']) == 0:
            synopse_articles_t_v3_article_groups_l2_insert_input_loc.append({
                "articles_group": response['initial_group'],
                'articles_in_group': len(response['initial_group'])
                }
                )
            synopse_articles_t_v1_rss1_articles_updates_loc.append({
                "where": {"id" : { "_eq": response['article_id'] }},
                "_set": {"is_grouped": 2}
                })
        if len(response_data1['data']['synopse_articles_t_v3_article_groups_l2']) == 1:
            articles  = []
            articles.append(response['initial_group'])
            articles.append(response_data1['data']['synopse_articles_t_v3_article_groups_l2'][0]['articles_group'])
            new_lst = []
            for sublist in articles:
                for element in sublist:
                    new_lst.append(element)
            articles = list(set(new_lst))
            synopse_articles_t_v3_article_groups_l2_updates_loc.append({
                "where": {"id" : { "_eq": response_data1['data']['synopse_articles_t_v3_article_groups_l2'][0]['id'] }},
                "_set": {"articles_group": articles, 'articles_in_group': len(articles)}
                })
            synopse_articles_t_v1_rss1_articles_updates_loc.append({
                "where": {"id" : { "_eq": response['article_id'] }},
                "_set": {"is_grouped": 2}
                })
    mutation_variables = {
        "objects": synopse_articles_t_v3_article_groups_l2_insert_input_loc,
        "updates": synopse_articles_t_v1_rss1_articles_updates_loc,
        "updates1": synopse_articles_t_v3_article_groups_l2_updates_loc,
        }
    out1 = mutation_hasura_graphql(endpoint=endpoint, admin_key=admin_key, mutation_query=mutation_query, mutation_variables=mutation_variables)
    
    

{'article_id': 449, 'initial_group': [449, 285, 108]}
{'data': {'insert_synopse_articles_t_v3_article_groups_l2': {'affected_rows': 1}, 'update_synopse_articles_t_v1_rss1_articles_many': [{'affected_rows': 1}], 'update_synopse_articles_t_v3_article_groups_l2_many': {'affected_rows': 0}}}
{'article_id': 443, 'initial_group': [443, 285, 296, 109, 94]}
{'data': {'insert_synopse_articles_t_v3_article_groups_l2': {'affected_rows': 1}, 'update_synopse_articles_t_v1_rss1_articles_many': [{'affected_rows': 1}], 'update_synopse_articles_t_v3_article_groups_l2_many': {'affected_rows': 0}}}
{'article_id': 458, 'initial_group': [502, 458]}
{'data': {'insert_synopse_articles_t_v3_article_groups_l2': {'affected_rows': 0}, 'update_synopse_articles_t_v1_rss1_articles_many': [{'affected_rows': 1}], 'update_synopse_articles_t_v3_article_groups_l2_many': [{'affected_rows': 1}]}}
{'article_id': 504, 'initial_group': [504, 441]}
{'data': {'insert_synopse_articles_t_v3_article_groups_l2': {'affected_rows': 

In [12]:
graphql_query = '''
query MyQuery($limit: Int!, $offset: Int!) {
  synopse_articles_t_v3_article_groups_l1(where: {t_v1_rss1_article: {is_grouped: {_eq: 1}}, article_count: {_gt: 1}}, order_by: {updated_at: asc}, limit: $limit, offset: $offset) {
    article_id
    initial_group
  }
}
'''
query2 = '''
query MyQuery($articleid: [bigint!] = []) {
    synopse_articles_t_v3_article_groups_l2(where: {articles_group: {_contains: $articleid}, is_valid: {_eq: 0}}, order_by: {updated_at: desc}) {
    id
    articles_group
    is_summerized
    is_valid
    }
}
'''
query3 = '''
query MyQuery($articleid: [bigint!] = []) {
    synopse_articles_t_v3_article_groups_l2(where: {articles_group: {_contains: $articleid}, is_valid: {_eq: 1}}, order_by: {updated_at: desc}, limit: 1) {
    id
    articles_group
    is_summerized
    is_valid
    }
}
'''
mutation_query = """
mutation MyMutation($objects: [synopse_articles_t_v3_article_groups_l2_insert_input!] = {}, $updates: [synopse_articles_t_v1_rss1_articles_updates!] = {where: {}}, $updates1: [synopse_articles_t_v3_article_groups_l2_updates!] = {where: {}}) {
    insert_synopse_articles_t_v3_article_groups_l2(objects: $objects, on_conflict: {constraint: t_v3_article_groups_l2_pkey}) {
    affected_rows
    }
    update_synopse_articles_t_v1_rss1_articles_many(updates: $updates) {
    affected_rows
    }
    update_synopse_articles_t_v3_article_groups_l2_many(updates: $updates1) {
    affected_rows
    }
}
"""
while True: 
    variables = {
    "limit": 1,
    "offset": 0
    }
    synopse_articles_t_v3_article_groups_l2_insert_input_loc=[]
    synopse_articles_t_v1_rss1_articles_updates_loc=[]
    synopse_articles_t_v3_article_groups_l2_updates_loc=[]
    response_data = query_hasura_graphql(endpoint, admin_key, graphql_query, variables)
    print(response_data)
    if len(response_data['data']['synopse_articles_t_v3_article_groups_l1']) == 0:
        break
    for response in response_data['data']['synopse_articles_t_v3_article_groups_l1']:
        initial_group = response['initial_group']
        new_article_ids = []
        ids = []
        for article in initial_group:
            variables2 = {
              "articleid": [article]
              }
            response_data1 = query_hasura_graphql(endpoint, admin_key, query2, variables2)
            if len(response_data1['data']['synopse_articles_t_v3_article_groups_l2']) > 0:
                for response1 in response_data1['data']['synopse_articles_t_v3_article_groups_l2']:
                    ids.append(response1['id'])
                    new_article_ids.append(response1['articles_group'])
        ids_exists = []
        for article in initial_group:
            variables2 = {
              "articleid": [article]
              }
            response_data1 = query_hasura_graphql(endpoint, admin_key, query3, variables2)
            if len(response_data1['data']['synopse_articles_t_v3_article_groups_l2']) > 0:
                for response1 in response_data1['data']['synopse_articles_t_v3_article_groups_l2']:
                    ids_exists.append(response1['id'])
                    new_article_ids.append(response1['articles_group'])
            
        ids = list(set(ids))
        if len(ids) == 0:
            if len(ids_exists) == 0:
                synopse_articles_t_v3_article_groups_l2_insert_input_loc.append({
                    "articles_group": response['initial_group'],
                    'articles_in_group': len(response['initial_group'])
                    }
                    )
                synopse_articles_t_v1_rss1_articles_updates_loc.append({
                    "where": {"id" : { "_eq": response['article_id'] }},
                    "_set": {"is_grouped": 2}
                    })
            elif len(ids_exists) > 0:
                new_lst = []
                new_article_ids.append(initial_group)
                for sublist in new_article_ids:
                    for element in sublist:
                        new_lst.append(element)
                new_article_ids = list(set(new_lst))
                synopse_articles_t_v3_article_groups_l2_insert_input_loc.append({
                    "articles_group": new_article_ids,
                    'articles_in_group': len(new_article_ids)
                    }
                    )
                synopse_articles_t_v1_rss1_articles_updates_loc.append({
                    "where": {"id" : { "_eq": response['article_id'] }},
                    "_set": {"is_grouped": 2}
                    })
        if len(ids) == 1:
            new_lst = []
            new_article_ids.append(initial_group)
            for sublist in new_article_ids:
                for element in sublist:
                    new_lst.append(element)
            new_article_ids = list(set(new_lst))
            synopse_articles_t_v3_article_groups_l2_updates_loc.append({
                "where": {"id" : { "_eq": ids[0] }},
                "_set": {"articles_group": new_article_ids, 'articles_in_group': len(new_article_ids)}
                })
            synopse_articles_t_v1_rss1_articles_updates_loc.append({
                "where": {"id" : { "_eq": response['article_id'] }},
                "_set": {"is_grouped": 2}
                })

    mutation_variables = {
        "objects": synopse_articles_t_v3_article_groups_l2_insert_input_loc,
        "updates": synopse_articles_t_v1_rss1_articles_updates_loc,
        "updates1": synopse_articles_t_v3_article_groups_l2_updates_loc,
        }
    out1 = mutation_hasura_graphql(endpoint=endpoint, admin_key=admin_key, mutation_query=mutation_query, mutation_variables=mutation_variables)
    
    
        

{'data': {'synopse_articles_t_v3_article_groups_l1': [{'article_id': 284, 'initial_group': [500, 284, 305, 109]}]}}
{'data': {'insert_synopse_articles_t_v3_article_groups_l2': {'affected_rows': 0}, 'update_synopse_articles_t_v1_rss1_articles_many': {'affected_rows': 0}, 'update_synopse_articles_t_v3_article_groups_l2_many': {'affected_rows': 0}}}
{'data': {'synopse_articles_t_v3_article_groups_l1': [{'article_id': 284, 'initial_group': [500, 284, 305, 109]}]}}
{'data': {'insert_synopse_articles_t_v3_article_groups_l2': {'affected_rows': 0}, 'update_synopse_articles_t_v1_rss1_articles_many': {'affected_rows': 0}, 'update_synopse_articles_t_v3_article_groups_l2_many': {'affected_rows': 0}}}
{'data': {'synopse_articles_t_v3_article_groups_l1': [{'article_id': 284, 'initial_group': [500, 284, 305, 109]}]}}


KeyboardInterrupt: 

In [30]:
graphql_query = '''
query MyQuery($limit: Int!, $offset: Int!) {
  synopse_articles_t_v3_article_groups_l1(where: {t_v1_rss1_article: {is_grouped: {_eq: 1}}, article_count: {_gt: 1}}, order_by: {updated_at: asc}, limit: $limit, offset: $offset) {
    article_id
    initial_group
  }
}
'''
query2 = '''
query MyQuery($articleid: [bigint!] = []) {
    synopse_articles_t_v3_article_groups_l1(where: {initial_group: {_contains: $articleid}}) {
    article_id
    initial_group
    }
}
'''
query3 = '''
query MyQuery($articleid: [bigint!] = []) {
    synopse_articles_t_v3_article_groups_l2(where: {articles_group: {_contains: $articleid}, is_valid: {_eq: 0}}, order_by: {is_valid: desc, updated_at: desc}, limit: 1) {
    id
    articles_group
    is_summerized
    is_valid
    }
}
'''
mutation_query = """
    mutation MyMutation($objects: [synopse_articles_t_v3_article_groups_l2_insert_input!] = {}, $updates: [synopse_articles_t_v1_rss1_articles_updates!] = {where: {}}, $updates1: [synopse_articles_t_v3_article_groups_l2_updates!] = {where: {}}) {
        insert_synopse_articles_t_v3_article_groups_l2(objects: $objects, on_conflict: {constraint: t_v3_article_groups_l2_pkey}) {
        affected_rows
        }
        update_synopse_articles_t_v1_rss1_articles_many(updates: $updates) {
        affected_rows
        }
        update_synopse_articles_t_v3_article_groups_l2_many(updates: $updates1) {
        affected_rows
        }
    }
"""
variables = {
"limit": 2,
"offset": 0
}
response_data = query_hasura_graphql(endpoint, admin_key, graphql_query, variables)
for response in response_data['data']['synopse_articles_t_v3_article_groups_l1']:
    print(response)
    variables2 = {
        "articleid": [response['article_id']]
        }
    synopse_articles_t_v3_article_groups_l2_insert_input_loc=[]
    synopse_articles_t_v1_rss1_articles_updates_loc=[]
    synopse_articles_t_v3_article_groups_l2_updates_loc=[]
    response_data1 = query_hasura_graphql(endpoint, admin_key, query2, variables2)
    articles_ids = []
    if len(response_data1['data']['synopse_articles_t_v3_article_groups_l1']) > 0:
        for func_response in response_data1['data']['synopse_articles_t_v3_article_groups_l1']:
            articles_ids.append(func_response['initial_group'])
    n1 = []
    for sublist in articles_ids:
        for element in sublist:
            n1.append(element)
    articles_ids = list(set(n1))
    articles_ids.sort(reverse=True)
    print(articles_ids)
    while True:
        if len(articles_ids) > 1:
            articles_news = []
            for article_id in articles_ids:
                variables3 = {
                    "articleid": [article_id]
                    }
                response_data2 = query_hasura_graphql(endpoint, admin_key, query2, variables3)
                if len(response_data1['data']['synopse_articles_t_v3_article_groups_l1']) > 0:
                    for func_response in response_data1['data']['synopse_articles_t_v3_article_groups_l1']:
                        articles_news.append(func_response['initial_group'])
            n2 = []
            for sublist in articles_news:
                for element in sublist:
                    n2.append(element)
            articles_news = list(set(n2))
            articles_news.sort(reverse=True)
            if articles_ids == articles_news:
                print(articles_news)
                break
            else:
                articles_ids = articles_news
    exists = 0
    ids = []
    articles_groups = []
    for article_id in articles_ids:
        variables4 = {
        "articleid": [article_id]
        }
        response_data3 = query_hasura_graphql(endpoint, admin_key, query3, variables4)
        if len(response_data3['data']['synopse_articles_t_v3_article_groups_l2']) > 0:
            for func_response in response_data3['data']['synopse_articles_t_v3_article_groups_l2']:
                if func_response['is_valid'] == 0:
                    exists = exists + 1
                    ids.append(func_response['id'])
                    articles_groups.append(func_response['articles_group'])
                
                    
    if exists == 0 : 
        synopse_articles_t_v3_article_groups_l2_insert_input_loc.append({
            "articles_group": articles_ids,
            'articles_in_group': len(articles_ids)
            }
            )
        synopse_articles_t_v1_rss1_articles_updates_loc.append({
            "where": {"id" : { "_eq": response['id'] }},
            "_set": {"is_grouped": 2}
            })

{'article_id': 502}
[502, 458, 416, 105]
[502, 458, 416, 105]
{'article_id': 449}
[449, 443, 349, 296, 285, 283, 116, 108, 94, 75]
[449, 443, 349, 296, 285, 283, 116, 108, 94, 75]


In [23]:
import json
graphql_query = '''
query MyQuery($limit: Int!, $offset: Int!) {
  synopse_articles_t_v3_article_groups_l1(where: {t_v1_rss1_article: {is_grouped: {_eq: 1}}, article_count: {_gt: 1}}, order_by: {updated_at: asc}, limit: $limit, offset: $offset) {
    article_id
    initial_group
  }
}
'''
query2 = '''
query MyQuery($articleid: [bigint!] = []) {
    synopse_articles_t_v3_article_groups_l1(where: {initial_group: {_contains: $articleid}}) {
    article_id
    initial_group
    }
}
'''
query3 = '''
query MyQuery($articleid: [bigint!] = [1088]) {
    synopse_articles_t_v3_article_groups_l2(where: {articles_group: {_contains: $articleid}, is_valid: {_eq: 0}}, order_by: {is_valid: desc, updated_at: desc}, limit: 1) {
    id
    articles_group
    is_summerized
    is_valid
    }
}
'''
offset = 10
while True:
    variables = {
    "limit": 2,
    "offset": offset
    }
    response_data = query_hasura_graphql(endpoint, admin_key, graphql_query, variables)
    if len(response_data['data']['synopse_articles_t_v3_article_groups_l1']) == 0:
        break
    #print(json.dumps(response_data, indent=4))
    for response in response_data['data']['synopse_articles_t_v3_article_groups_l1']:
        variables2 = {
            "articleid": [response['article_id']]
            }
        response_data1 = query_hasura_graphql(endpoint, admin_key, query2, variables2)
        articles_ids = []
        if len(response_data1['data']['synopse_articles_t_v3_article_groups_l1']) > 0:
            for func_response in response_data1['data']['synopse_articles_t_v3_article_groups_l1']:
                articles_ids.append(func_response['initial_group'])
        n1 = []
        for sublist in articles_ids:
            for element in sublist:
                n1.append(element)
        articles_ids = list(set(n1))
        articles_ids.sort(reverse=True)
        while True:
            if len(articles_ids) > 1:
                articles_news = []
                for article_id in articles_ids:
                    variables3 = {
                        "articleid": [article_id]
                        }
                    response_data2 = query_hasura_graphql(endpoint, admin_key, query2, variables3)
                    if len(response_data1['data']['synopse_articles_t_v3_article_groups_l1']) > 0:
                        for func_response in response_data1['data']['synopse_articles_t_v3_article_groups_l1']:
                            articles_news.append(func_response['initial_group'])
                n2 = []
                for sublist in articles_news:
                    for element in sublist:
                        n2.append(element)
                articles_news = list(set(n2))
                articles_news.sort(reverse=True)
                if articles_ids == articles_news:
                    break
                else:
                    articles_ids = articles_news
        
        
    break

[502, 416]
[502, 458, 416, 105]
[502, 458, 416, 105]
[502, 458, 416, 105]
##############
[403, 109]
[443, 403, 284, 111, 109]
[443, 403, 284, 111, 109]
[443, 403, 284, 111, 109]
##############


In [21]:
for response in response_data['data']['synopse_articles_t_v3_article_groups_l1']:
    print(response['initial_group'])
    variables2 = {
        "articleid": [response['article_id']]
        }
    response_data1 = query_hasura_graphql(endpoint, admin_key, query2, variables2)
    articles_ids = []
    if len(response_data1['data']['synopse_articles_t_v3_article_groups_l1']) > 0:
        for func_response in response_data1['data']['synopse_articles_t_v3_article_groups_l1']:
            articles_ids.append(func_response['initial_group'])
    n1 = []
    for sublist in articles_ids:
        for element in sublist:
            n1.append(element)
    articles_ids = list(set(n1))
    articles_ids.sort()  # Sort the list in-place
    print(articles_ids)
    while True:
        if len(articles_ids) > 1:
            articles_news = []
            for article_id in articles_ids:
                variables3 = {
                    "articleid": [article_id]
                    }
                response_data2 = query_hasura_graphql(endpoint, admin_key, query2, variables3)
                if len(response_data1['data']['synopse_articles_t_v3_article_groups_l1']) > 0:
                    for func_response in response_data1['data']['synopse_articles_t_v3_article_groups_l1']:
                        articles_news.append(func_response['initial_group'])
            n2 = []
            print(articles_news)
            for sublist in articles_news:
                for element in sublist:
                    n2.append(element)
            articles_news = list(set(n2))
            articles_news.sort()  # Sort the list in-place
            print(articles_news)
            break
    print("###########")

[413, 51]
[51, 413]
[[413, 51], [413, 51], [413, 51], [413, 51]]
[51, 413]
###########
[502, 416]
[105, 416, 458, 502]
[[502, 458, 416, 105], [502, 416], [502, 458, 416, 105], [502, 416], [502, 458, 416, 105], [502, 416], [502, 458, 416, 105], [502, 416]]
[105, 416, 458, 502]
###########
[403, 109]
[109, 111, 284, 403, 443]
[[403, 109], [443, 403, 284, 109, 111], [403, 109], [443, 403, 284, 109, 111], [403, 109], [443, 403, 284, 109, 111], [403, 109], [443, 403, 284, 109, 111], [403, 109], [443, 403, 284, 109, 111]]
[109, 111, 284, 403, 443]
###########
[287, 77, 61]
[61, 77, 287]
[[287, 77, 61], [287, 77], [287, 61], [287, 77, 61], [287, 77], [287, 61], [287, 77, 61], [287, 77], [287, 61]]
[61, 77, 287]
###########
[282, 299]
[282, 299]
[[282, 299], [282, 299], [282, 299], [282, 299]]
[282, 299]
###########


In [11]:
import json
graphql_query = '''
query MyQuery($limit: Int!, $offset: Int!) {
  synopse_articles_t_v1_rss1_articles(limit: $limit, offset: $offset, order_by: {created_at: desc}, where: { id: {_in: [443, 403, 284, 109, 111,  285, 296, 94 ]}}) {
    id
    t_v3_article_groups_l1 {
      initial_group
    }
  }
}
'''
query2 = '''
query MyQuery($articleid: [bigint!] = [1088]) {
    synopse_articles_t_v3_article_groups_l1(where: {initial_group: {_contains: $articleid}}) {
    initial_group
    }
}
'''
query3 = '''
query MyQuery($articleid: [bigint!] = [1088]) {
  synopse_articles_t_v3_article_groups_l2(where: {articles_group: {_contains: $articleid}, is_valid: {_eq: 0}}, order_by: {is_valid: desc, updated_at: desc}, limit: 1) {
    id
    articles_group
    is_summerized
    is_valid
  }
}
'''
offset = 0
variables = {
"limit": 20,
"offset": offset
}
mutation_query = """
    mutation MyMutation($objects: [synopse_articles_t_v3_article_groups_l2_insert_input!] = {}, $updates: [synopse_articles_t_v1_rss1_articles_updates!] = {where: {}}, $updates1: [synopse_articles_t_v3_article_groups_l2_updates!] = {where: {}}) {
        insert_synopse_articles_t_v3_article_groups_l2(objects: $objects, on_conflict: {constraint: t_v3_article_groups_l2_pkey}) {
        affected_rows
        }
        update_synopse_articles_t_v1_rss1_articles_many(updates: $updates) {
        affected_rows
        }
        update_synopse_articles_t_v3_article_groups_l2_many(updates: $updates1) {
        affected_rows
        }
    }
"""
response_data = query_hasura_graphql(endpoint, admin_key, graphql_query, variables)
#print(json.dumps(response_data, indent=4))
for response in response_data['data']['synopse_articles_t_v1_rss1_articles']:
    variables2 = {
        "articleid": [response['id']]
        }
    synopse_articles_t_v3_article_groups_l2_insert_input_loc=[]
    synopse_articles_t_v1_rss1_articles_updates_loc=[]
    synopse_articles_t_v3_article_groups_l2_updates_loc=[]
    func_response_data = query_hasura_graphql(endpoint, admin_key, query2, variables2)
    #print(json.dumps(func_response_data, indent=4))
    articles_ids = []
    if len(func_response_data['data']['synopse_articles_t_v3_article_groups_l1']) > 0:
        for func_response in func_response_data['data']['synopse_articles_t_v3_article_groups_l1']:
            articles_ids.append(func_response['initial_group'])
    print(articles_ids)
    for article in articles_ids:
        variables3 = {
            "articleid": [article]
            }
        func_response_data1 = query_hasura_graphql(endpoint, admin_key, query3, variables3)n
    if len(func_response_data1['data']['synopse_articles_t_v3_article_groups_l2']) == 0:
        print("No Data")
        new_lst = []
        for sublist in articles_ids:
            for element in sublist:
                new_lst.append(element)
        my_list = list(set(new_lst))
        synopse_articles_t_v3_article_groups_l2_insert_input_loc.append({
            "articles_group": my_list,
            'articles_in_group': len(my_list)
            }
            )
        synopse_articles_t_v1_rss1_articles_updates_loc.append({
            "where": {"id" : { "_eq": response['id'] }},
            "_set": {"is_grouped": 2}
            })
    elif len(func_response_data1['data']['synopse_articles_t_v3_article_groups_l2']) == 1:
        articles_ids.append(func_response_data1['data']['synopse_articles_t_v3_article_groups_l2'][0]['articles_group'])
        new_lst = []
        for sublist in articles_ids:
            for element in sublist:
                new_lst.append(element)
        my_list = list(set(new_lst))
        synopse_articles_t_v3_article_groups_l2_updates_loc.append({
            "where": {"id" : { "_eq": func_response_data1['data']['synopse_articles_t_v3_article_groups_l2'][0]['id'] }},
            "_set": {"articles_group": my_list, 'articles_in_group': len(my_list)}
            })
        synopse_articles_t_v1_rss1_articles_updates_loc.append({
            "where": {"id" : { "_eq": response['id'] }},
            "_set": {"is_grouped": 2}
            })
        

mutation_variables = {
"objects": synopse_articles_t_v3_article_groups_l2_insert_input_loc,
"updates": synopse_articles_t_v1_rss1_articles_updates_loc,
"updates1": synopse_articles_t_v3_article_groups_l2_updates_loc,
}
out1 = mutation_hasura_graphql(endpoint=endpoint, admin_key=admin_key, mutation_query=mutation_query, mutation_variables=mutation_variables)


[[443, 285, 296, 109, 94], [443, 449, 283, 285, 349, 296, 75, 94], [443, 285, 296, 94], [443, 285, 296, 94], [443, 403, 284, 109, 111]]
[[403, 109], [443, 403, 284, 109, 111]]
[[443, 285, 296, 109, 94], [443, 449, 283, 285, 349, 296, 75, 94], [443, 285, 296, 94], [443, 285, 296, 94]]
[[443, 285, 296, 109, 94], [449, 285, 108], [283, 285], [285, 349], [443, 449, 283, 285, 349, 296, 75, 94], [443, 285, 296, 94], [285, 75], [443, 285, 296, 94]]
[[500, 284, 305], [500, 284, 305, 109], [500, 284, 305], [443, 403, 284, 109, 111]]
[[443, 285, 296, 109, 94], [443, 449, 283, 285, 349, 296, 75, 94], [443, 285, 296, 94], [443, 285, 296, 94]]
[[443, 285, 296, 109, 94], [403, 109], [500, 284, 305, 109], [109, 111], [443, 403, 284, 109, 111]]
[[109, 111], [443, 403, 284, 109, 111]]
{'data': {'insert_synopse_articles_t_v3_article_groups_l2': {'affected_rows': 0}, 'update_synopse_articles_t_v1_rss1_articles_many': [{'affected_rows': 1}], 'update_synopse_articles_t_v3_article_groups_l2_many': [{'affect

In [25]:
def grouping_l2(offset1):
    graphql_query = '''
      query MyQuery($limit: Int!, $offset: Int!) {
        synopse_articles_t_v3_article_groups_l1(where: {t_v1_rss1_article: {is_grouped: {_eq: 1}}, article_count: {_gt: 1}}, order_by: {updated_at: asc}, limit: $limit, offset: $offset) {
          article_id
        }
      }
      '''
    offset = offset1
    mutation_query = """
        mutation MyMutation($objects: [synopse_articles_t_v3_article_groups_l2_insert_input!] = {}, $updates: [synopse_articles_t_v1_rss1_articles_updates!] = {where: {}}, $updates1: [synopse_articles_t_v3_article_groups_l2_updates!] = {where: {}}) {
          insert_synopse_articles_t_v3_article_groups_l2(objects: $objects, on_conflict: {constraint: t_v3_article_groups_l2_pkey}) {
            affected_rows
          }
          update_synopse_articles_t_v1_rss1_articles_many(updates: $updates) {
            affected_rows
          }
          update_synopse_articles_t_v3_article_groups_l2_many(updates: $updates1) {
            affected_rows
          }
        }
    """
    query2 = '''
    query MyQuery($articleid: [bigint!] = [1088]) {
        synopse_articles_t_v3_article_groups_l1(where: {initial_group: {_contains: $articleid}}) {
        initial_group
        }
    }
    '''
    query3 = '''
    query MyQuery($articleid: [bigint!] = [1088]) {
      synopse_articles_t_v3_article_groups_l2(where: {articles_group: {_contains: $articleid}, is_valid: {_eq: 0}}, order_by: {is_valid: desc, updated_at: desc}, limit: 1) {
        id
        articles_group
        is_summerized
        is_valid
      }
    }
    '''
    while True:
        variables = {
        "limit": 20,
        "offset": offset
        }
        synopse_articles_t_v3_article_groups_l2_insert_input_loc=[]
        synopse_articles_t_v1_rss1_articles_updates_loc=[]
        synopse_articles_t_v3_article_groups_l2_updates_loc=[]
        response_data = query_hasura_graphql(endpoint, admin_key, graphql_query, variables)
        if len(response_data['data']['synopse_articles_t_v3_article_groups_l1']) == 0:
            break
        #print(json.dumps(response_data, indent=4))
        for response in response_data['data']['synopse_articles_t_v3_article_groups_l1']:
            variables3 = {
                "articleid": [response['article_id']]
                }
            func_response_data = query_hasura_graphql(endpoint, admin_key, query2, variables3)
            #print(json.dumps(func_response_data, indent=4))
            articles_ids = []
            if len(func_response_data['data']['synopse_articles_t_v3_article_groups_l1']) > 0:
                for func_response in func_response_data['data']['synopse_articles_t_v3_article_groups_l1']:
                    articles_ids.append(func_response['initial_group'])
            func_response_data1 = query_hasura_graphql(endpoint, admin_key, query3, variables3)
            if len(func_response_data1['data']['synopse_articles_t_v3_article_groups_l2']) == 0:
                new_lst = []
                for sublist in articles_ids:
                    for element in sublist:
                        new_lst.append(element)
                my_list = list(set(new_lst))
                synopse_articles_t_v3_article_groups_l2_insert_input_loc.append({
                    "articles_group": my_list,
                    'articles_in_group': len(my_list)
                    }
                    )
                synopse_articles_t_v1_rss1_articles_updates_loc.append({
                    "where": {"id" : { "_eq": response['article_id'] }},
                    "_set": {"is_grouped": 2}
                    })
            elif len(func_response_data1['data']['synopse_articles_t_v3_article_groups_l2']) == 1:
                articles_ids.append(func_response_data1['data']['synopse_articles_t_v3_article_groups_l2'][0]['articles_group'])
                new_lst = []
                for sublist in articles_ids:
                    for element in sublist:
                        new_lst.append(element)
                my_list = list(set(new_lst))
                synopse_articles_t_v3_article_groups_l2_updates_loc.append({
                    "where": {"id" : { "_eq": func_response_data1['data']['synopse_articles_t_v3_article_groups_l2'][0]['id'] }},
                    "_set": {"articles_group": my_list, 'articles_in_group': len(my_list)}
                    })
                synopse_articles_t_v1_rss1_articles_updates_loc.append({
                    "where": {"id" : { "_eq": response['article_id'] }},
                    "_set": {"is_grouped": 2}
                    })


        mutation_variables = {
        "objects": synopse_articles_t_v3_article_groups_l2_insert_input_loc,
        "updates": synopse_articles_t_v1_rss1_articles_updates_loc,
        "updates1": synopse_articles_t_v3_article_groups_l2_updates_loc,
        }
        out1 = mutation_hasura_graphql(endpoint=endpoint, admin_key=admin_key, mutation_query=mutation_query, mutation_variables=mutation_variables)
grouping_l2(0)

{'data': {'insert_synopse_articles_t_v3_article_groups_l2': {'affected_rows': 20}, 'update_synopse_articles_t_v1_rss1_articles_many': [{'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}], 'update_synopse_articles_t_v3_article_groups_l2_many': {'affected_rows': 0}}}
{'data': {'insert_synopse_articles_t_v3_article_groups_l2': {'affected_rows': 6}, 'update_synopse_articles_t_v1_rss1_articles_many': [{'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_rows': 1}, {'affected_row

In [4]:
mutation_query = """
mutation MyMutation($objects: [synopse_articles_t_v4_tags_hierarchy_insert_input!] = {}) {
  insert_synopse_articles_t_v4_tags_hierarchy(objects: $objects, on_conflict: {constraint: t_v4_tags_hierarchy_pkey}) {
    affected_rows
  }
}
"""
synopse_articles_t_v4_tags_hierarchy_insert_input_loc=[]
news_subcategories =  [
    ['Politics', 'Coverage of political events, government policies, elections, and international relations'],
    ['Business', 'Reports on financial markets, companies, mergers and acquisitions, and economic trends'],
    ['Science and Technology', 'News related to scientific discoveries, innovations, and technological advancements'],
    ['Health and Medicine', 'Updates on medical research, healthcare policies, and public health issues'],
    ['Entertainment', 'Information on movies, music, celebrities, and the entertainment industry'],
    ['Sports', 'Coverage of sporting events, scores, athletes, and sports-related news'],
    ['Environment', 'Reports on climate change, conservation efforts, and environmental issues'],
    ['Human Interest', 'Stories about people, personal experiences, and inspirational narratives'],
]

for n in news_subcategories:
    synopse_articles_t_v4_tags_hierarchy_insert_input_loc.append({
        "tag": n[0],
        "tag_description": n[1]
    })
mutation_variables = {
"objects": synopse_articles_t_v4_tags_hierarchy_insert_input_loc,
}
out1 = mutation_hasura_graphql(endpoint=endpoint, admin_key=admin_key, mutation_query=mutation_query, mutation_variables=mutation_variables)


NameError: name 'mutation_hasura_graphql' is not defined

In [12]:
import spacy
from keybert import KeyBERT

nlp = spacy.load("en_core_web_trf")

In [27]:
graphql_tags_query = '''
query MyQuery {
  synopse_articles_t_v4_tags_hierarchy(where: {is_valid: {_eq: 1}}) {
    id
    tag
    tag_hierachy
  }
}
'''
tags_main = []
tags_ids = []
variables_tags = {
}
response_data = query_hasura_graphql(endpoint, admin_key, graphql_tags_query, variables_tags)
tags_main = []
for response in response_data['data']['synopse_articles_t_v4_tags_hierarchy']:
  if response['tag_hierachy'] == 0:
    tags_main.append(response['tag'])
    tags_ids.append(response['id'])
print(tags_main)
print(tags_ids)

None
None
{'data': {'update_synopse_articles_t_v2_articles_summary_many': [{'affected_rows': 1}, {'affected_rows': 1}], 'update_synopse_articles_t_v1_rss1_articles_many': [{'affected_rows': 1}, {'affected_rows': 1}]}}


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1  # Use GPU if available, else CPU
print(device)
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer, device=device)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

In [6]:
graphql_tags_query = '''
query MyQuery {
  synopse_articles_t_v4_tags_hierarchy(where: {is_valid: {_eq: 1}}) {
    id
    tag
    tag_hierachy
  }
}
'''
tags_main = []
tags_ids = []
variables_tags = {
}
response_data_tags = query_hasura_graphql(endpoint, admin_key, graphql_tags_query, variables_tags)
tags_main = []
for response in response_data_tags['data']['synopse_articles_t_v4_tags_hierarchy']:
  if response['tag_hierachy'] == 0:
    tags_main.append(response['tag'])
    tags_ids.append(response['id'])
print(tags_main)
print(tags_ids)
index = tags_main.index("Business")
print(index)
print(tags_ids[index])
tags_sub = []
for response in response_data_tags['data']['synopse_articles_t_v4_tags_hierarchy']:
  if response['tag_hierachy'] == tags_ids[index]:
    tags_sub.append(response['tag'])
print(tags_sub)

['Politics', 'Business', 'Technology', 'Science', 'Health', 'Sports', 'Entertainment', 'World', 'Lifestyle', 'Environment', 'Finance']
[274, 275, 276, 277, 278, 279, 280, 283, 284, 281, 290]
1
275
['Business', 'Crypto', 'Real Estate', 'Stocks', 'Investing']
